In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
cd ~/demres

/Users/zurfarosa/demres


In [76]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists,druglists
from demres.common.process_pt_features import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design as sd
from demres.demins.statistical_functions import *
from common.helper_functions import *

In [77]:
pd.set_option('display.max_columns', None)

## Process raw CSV files

In [5]:
# create_pegmed()

In [6]:
# create_pegprod()

In [7]:
# create_prescriptions()

In [8]:
# create_consultations()

In [9]:
# create_clinicals()

In [10]:
# create_tests()

In [11]:
# create_referrals()

In [12]:
# create_immunisations()

In [13]:
# create_medcoded_entries()

## Create basic pt_features dataframe
*pt_features will contain all the variables (e.g. age, male gender, insomnia count) used in the logistic regression*

In [8]:
# specify subtype of dementia we're interested in - 'vascular','alzheimers' or 'all_dementia'
subtype = 'all_dementia' 

In [9]:
all_entries = get_all_entries()

KeyboardInterrupt: 

In [ ]:
pt_features = create_pt_features()

In [ ]:
pt_features = get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries,pt_features)

In [ ]:
pt_features = only_include_specific_dementia_subtype(pt_features,subtype=subtype)

In [ ]:
pt_features = add_data_start_and_end_dates(all_entries,pt_features)

In [ ]:
len(pt_features[pt_features['isCase']==True]),len(pt_features[pt_features['isCase']==False])

In [ ]:
pt_features.head(3)

In [ ]:
pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',index=False)

In [ ]:
for window in sd.exposure_windows:
    print(window['name'],' being matched')    
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    pt_features = match_cases_and_controls(pt_features,req_yrs_post_index=sd.req_yrs_post_index,start_year=abs(window['start_year']))
    pt_features = delete_unmatched_cases_and_controls(pt_features)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

## Add derived variables to pt_features 
*e.g. insomnia count, presence of diabetes, consultation count*

In [10]:
medcoded_entries = pd.read_hdf('hdf/medcoded_entries.hdf')

In [36]:
prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [93]:
# Add condition status (e.g. insomnia count, presence of diabetes, presence of stroke)
for window in sd.exposure_windows:
    print(window['name'],'...')
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    pt_features.drop(['insomnia_count:0', 'insomnia_count:1_10',
       'insomnia_count:above_10', 'non_insomnia_GP_consultations:0','insomnia', 'insomnia_any','insomnia_count'],axis=1,inplace=True)
    pt_features = get_multiple_condition_statuses(pt_features,medcoded_entries,window,[codelists.insomnia])
#     pt_features = create_pdds(pt_features,prescriptions,window,druglists.all_druglists)
#     pt_features = get_consultation_count(pt_features,all_entries,window)
    pt_features = create_quantiles_and_booleans(pt_features)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)


12_to_7 ...
insomnia
	Total insomnia events in all medcoded_events dataframe: 60453
	insomnia is being measured only during the window period
	insomnia events in this window for our patients: 3132
	Unique values   {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 24, 25, 58}
10_to_5 ...
insomnia
	Total insomnia events in all medcoded_events dataframe: 60453
	insomnia is being measured only during the window period
	insomnia events in this window for our patients: 2717
	Unique values   {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 53}
8_to_3 ...
insomnia
	Total insomnia events in all medcoded_events dataframe: 60453
	insomnia is being measured only during the window period
	insomnia events in this window for our patients: 2060
	Unique values   {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 38, 13, 14, 15, 16}


In [94]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'non_insomnia_GP_consultations', 'stroke',
       'CHD_heart_failure_and_peripheral_vascular_disease', 'hypertension',
       'diabetes', 'mental_illness_non_smi', 'mental_illness_smi',
       'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy',
       'age_at_index_date:65-69', 'age_at_index_date:70-74',
       'age_at_index_date:75-79', 'age_at_index_date:80-84',
       'age_at_index_date:85-89', 'age_at_index_date:90-99',
       'age_at_index_date:above_99', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000', 'antidepressants_pdds:0',
       'antidepressants_pdds:1_10', 'antidepressants_pdds:11_100',
       'antidepressants_pdds:101_1000', 'antidepressants_pdds:1001_10000',
       'anti